In [3]:
import torch
import cv2
import numpy as np
from PIL import Image
import requests
import io
import onnxruntime as ort
# Set up ONNX Runtime session with DirectML
from ultralytics import YOLO
from scipy.spatial.distance import cdist
import threading


In [2]:
pip install ultralytics onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 601.9 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [4]:
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from IPython.display import clear_output
import cv2
from tqdm import tqdm

class TrafficVisualization3D:
    def __init__(self, map_size, road_network, car_model_path='benchi-ML500.json'):
        self.map_size = map_size
        self.road_network = road_network
        self.fig = plt.figure(figsize=(8, 8), dpi=100)
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.setup_plot()
        self.load_car_model(car_model_path)

    def load_car_model(self, car_model_path):
        with open(car_model_path) as json_file:
            data = json.load(json_file)
            self.car_vertices = np.array(data['vertices'])
            self.car_faces = np.array(data['faces']) - 1

        # Scale up the car model
        scale_factor = 8  # Increased scale factor
        self.car_vertices *= scale_factor

        # Rotate the car model to lie flat on the ground
        rotation_matrix = np.array([
            [1, 0, 0],
            [0, 0, -1],
            [0, 1, 0]
        ])
        self.car_vertices = np.dot(self.car_vertices, rotation_matrix)

    def setup_plot(self):
        self.ax.set_xlim(0, self.map_size[0])
        self.ax.set_ylim(0, self.map_size[1])
        self.ax.set_zlim(0, 50)  # Reduced z-limit for a better view
        self.ax.axis('off')

        # Set camera angle for a more immersive view
        # Set camera angle for 3D perspective view
        self.ax.view_init(elev=60, azim=80)
        #Change the azimuth of the map.


        for i in range(0, len(self.road_network), 4):
            road = self.road_network[i:i+4]
            x = road[:, 0]
            y = road[:, 1]
            z = np.zeros_like(x)
            verts = [list(zip(x, y, z))]
            self.ax.add_collection3d(Poly3DCollection(verts, facecolors='gray', alpha=0.5))

        # Add ground plane
        x = np.linspace(0, self.map_size[0], 2)
        y = np.linspace(0, self.map_size[1], 2)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        self.ax.plot_surface(X, Y, Z, color='green', alpha=0.2)

    def update(self, vehicles):
        for collection in self.ax.collections:
            collection.remove()

        # Redraw road network and ground plane
        self.setup_plot()

        # Draw cars
        for vehicle in vehicles[:10]:  # Increased to 10 cars
            x, y = vehicle['position']
            z = 0
            self.draw_car(x, y, z, color='lightblue')

        self.fig.canvas.draw()
        img = np.frombuffer(self.fig.canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape(self.fig.canvas.get_width_height()[::-1] + (3,))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        return img

    def draw_car(self, x, y, z, color='lightblue'):
        # Translate the car model to the correct position
        translated_vertices = self.car_vertices + np.array([x, y, z])

        # Create a list of vertex coordinates for each face
        mesh = Poly3DCollection([translated_vertices[face] for face in self.car_faces],
                                facecolors=color, edgecolors='k', linewidths=0.1, alpha=0.8)
        self.ax.add_collection3d(mesh)



In [8]:
import cv2
import numpy as np
import onnxruntime as ort
from scipy.spatial.distance import cdist
from collections import deque
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from IPython.display import display, HTML
from base64 import b64encode
from google.colab import files
from tqdm.notebook import tqdm

class RunDetection2:
    def __init__(self, video_path, model_path='yolov5su.onnx'):
        self.video_path = video_path
        self.map_size = (400, 400)
        self.cap = cv2.VideoCapture(video_path)

        plt.ion()  # Turn on interactive mode for real-time plotting

        # Define the providers, if you're using CUDA replace AzureExecutionProvider... with CUDAExecutionProvider
        providers = ['AzureExecutionProvider', 'CPUExecutionProvider']
        self.session = ort.InferenceSession(model_path, providers=providers)

        # Generate road network with two thick vertical lines
        self.road_network = self.generate_vertical_roads()
        self.visualizer = TrafficVisualization3D(self.map_size, self.road_network)

        # Define source and destination points for homography
        self.src_points = np.array([
            [0, self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)],
            [self.cap.get(cv2.CAP_PROP_FRAME_WIDTH), self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)],
            [self.cap.get(cv2.CAP_PROP_FRAME_WIDTH), 0],
            [0, 0]
        ], dtype=np.float32)

        self.dst_points = np.array([
            [0, self.map_size[1]],
            [self.map_size[0], self.map_size[1]],
            [self.map_size[0], 0],
            [0, 0]
        ], dtype=np.float32)




        # Calculate homography matrix
        self.H = cv2.getPerspectiveTransform(self.src_points, self.dst_points)

        # Load car sprite
        # self.car_sprite = cv2.imread(car_png, cv2.IMREAD_UNCHANGED)
        # self.car_sprite = cv2.resize(self.car_sprite, (40, 80))  # Increased size

        # Dictionary to store previous positions of cars
        self.prev_positions = {}

    def generate_vertical_roads(self):
        road_width = 220  # Width of the vertical roads
        center_x = self.map_size[0] // 2  # Center of the map
        road_network = []  # List to store road network points
        x1 = center_x - road_width // 2  # Start point of the first road
        road_network.extend([(x1, 0), (x1, self.map_size[1]),
         (x1 + road_width,
          self.map_size[1]),
           (x1 + road_width, 0)])
        return np.array(road_network, dtype=np.int32)  # Convert to NumPy array


    # def generate_vertical_roads(self):
    #       road_width = 70  # Increased road width
    #       road_spacing = 15 # Space between the two roads
    #       road_network = []

    #       # Calculate the center of the map
    #       center_x = self.map_size[0] // 2

    #     #


    #       # Left vertical road
    #       x1 = center_x - road_width - road_spacing // 2
    #       road_network.extend([
    #           [x1, 0],
    #           [x1, self.map_size[1]],
    #           [x1 + road_width, self.map_size[1]],
    #           [x1 + road_width, 0]
    #       ])

    #       # Right vertical road
    #       x2 = center_x + road_spacing // 2
    #       road_network.extend([
    #           [x2, 0],
    #           [x2, self.map_size[1]],
    #           [x2 + road_width, self.map_size[1]],
    #           [x2 + road_width, 0]
    #       ])

    #       return np.array(road_network, dtype=np.int32)  # Convert to NumPy array


    def process_output(self, output, img_size, conf_threshold=0.15, iou_threshold=0.35):
        outputs = np.transpose(output[0])
        rows = outputs.shape[0]
        boxes = []
        scores = []
        class_ids = []

        img_height, img_width = img_size
        x_factor = img_width / 640
        y_factor = img_height / 640

        for i in range(rows):
            classes_scores = outputs[i][4:]
            max_score = np.amax(classes_scores)
            if max_score >= conf_threshold:
                class_id = np.argmax(classes_scores)
                x, y, w, h = outputs[i][0], outputs[i][1], outputs[i][2], outputs[i][3]
                left = int((x - w/2) * x_factor)
                top = int((y - h/2) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                boxes.append([left, top, width, height])
                scores.append(max_score)
                class_ids.append(class_id)

        indices = cv2.dnn.NMSBoxes(boxes, scores, conf_threshold, iou_threshold)

        detections = []
        for i in indices:
            box = boxes[i]
            score = scores[i]
            class_id = class_ids[i]
            detections.append([box[0], box[1], box[0]+box[2], box[1]+box[3], score, class_id])

        return detections

    def project_to_map(self, point):
        px, py = point
        projected_point = np.dot(self.H, np.array([px, py, 1]))
        projected_point = projected_point / projected_point[2]
        return (int(projected_point[0]), int(projected_point[1]))

    def snap_to_road(self, point):
        distances = cdist([point], self.road_network)
        closest_segment_idx = np.argmin(distances)
        if closest_segment_idx % 2 == 0:
            start, end = self.road_network[closest_segment_idx], self.road_network[closest_segment_idx + 1]
        else:
            start, end = self.road_network[closest_segment_idx - 1], self.road_network[closest_segment_idx]

        line_vec = end - start
        point_vec = np.array(point) - start
        projection = np.dot(point_vec, line_vec) / np.dot(line_vec, line_vec)
        projection = np.clip(projection, 0, 1)

        snapped_point = start + projection * line_vec

        smoothing_factor = 0.6  # Adjust this value to control the smoothing effect
        smooth_point = tuple(map(int, np.array(point) * (1 - smoothing_factor) + np.array(snapped_point) * smoothing_factor))

        return smooth_point

    def get_direction(self, current_pos, prev_pos):
        if prev_pos is None:
            return 0

        dx = current_pos[0] - prev_pos[0]
        dy = current_pos[1] - prev_pos[1]
        angle = np.degrees(np.arctan2(dy, dx))
        return angle

    def rotate_sprite(self, sprite, angle):
        rows, cols = sprite.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        return cv2.warpAffine(sprite, M, (cols, rows), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_TRANSPARENT)


    #Use tqdm to process the videos
    def run(self, slowdown_factor=2):
        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_skip = 3  # Process every 4th frame
        max_cars = 10  # Maximum number of cars to display
        frame_count = 0
        print("Processing the video")

        # Get video properties
        width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(self.cap.get(cv2.CAP_PROP_FPS))

        # Create output video file with reduced fps
        output_filename = 'output_video_slow3.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_filename, fourcc, fps // slowdown_factor, (width, height))
        with tqdm(total=total_frames // frame_skip, desc="Processing Frames") as pbar:
          while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break

            frame_count += 1
            if frame_count % frame_skip != 0:
                continue

            # Preprocess the frame
            input_height, input_width = 640, 640 # Resize to 1280,1280
            img = cv2.resize(frame, (input_width, input_height))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.transpose((2, 0, 1)).astype(np.float32)
            img /= 255.0
            img = np.expand_dims(img, axis=0)

            # Run inference
            inputs = {self.session.get_inputs()[0].name: img}
            outputs = self.session.run(None, inputs)

            # Process the output
            detections = self.process_output(outputs[0], frame.shape[:2])

            frame_vehicles = []
            for detection in detections[:max_cars]:
                x1, y1, x2, y2, score, class_id = detection
                if class_id == 2:  # Assuming class ID 2 is for cars, adjust if needed
                    car_position = ((x1 + x2) // 2, y2)  # Bottom center of bounding box
                    map_position = self.project_to_map(car_position)
                    snapped_position = self.snap_to_road(map_position)
                    frame_vehicles.append({
                        'position': snapped_position,
                        'type': 'car',
                        'score': score
                    })

            # Update the visualization
            map_view = self.visualizer.update(frame_vehicles)

            # Resize the map view to fit in the corner
            map_view = cv2.resize(map_view, (frame.shape[1] // 3, frame.shape[0] // 3))

            # Create a region of interest (ROI) in the top left corner of each frame
            roi = frame[0:map_view.shape[0], 0:map_view.shape[1]]

            # Create a mask of the map view and its inverse mask
            map_gray = cv2.cvtColor(map_view, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(map_gray, 1, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            # Black-out the area of the map view in ROI
            roi_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of map view from map image
            map_fg = cv2.bitwise_and(map_view, map_view, mask=mask)

            # Put map view in ROI and modify the main image
            dst = cv2.add(roi_bg, map_fg)
            frame[0:map_view.shape[0], 0:map_view.shape[1]] = dst

            # Write the frame to the output video multiple times to create slow-motion effect
            for _ in range(slowdown_factor):
                out.write(frame)
            pbar.update(1)


            # Optional: Print progress
            if frame_count % 100 == 0:
                print(f"Processed {frame_count} frames")


        # Release resources
        self.cap.release()
        out.release()
        cv2.destroyAllWindows()

        print(f"Video processing complete. Slow-motion output saved as {output_filename}")
        return output_filename

    def process_video(self, slowdown_factor=2):
        output_filename = self.run(slowdown_factor)

        # Provide download link
        files.download(output_filename)

        # Display a message with instructions
        display(HTML(f"""
        <p>Video processing complete. The slow-motion output video has been saved as {output_filename}.</p>
        <p>If the download doesn't start automatically, please check your browser's download manager.</p>
        <p>You can also find the video file in the Colab file browser (folder icon in the left sidebar).</p>
        """))



In [ ]:
# Usage
#video
video = 'newvid4.mp4'
#model
model = 'yolov8l.onnx'
#detection
detector = RunDetection2(video,model)
#slow down the video
detector.process_video(slowdown_factor=2)

Processing the video


Processing Frames:   0%|          | 0/111 [00:00<?, ?it/s]

In [6]:
from ultralytics import YOLO
model = YOLO('yolov8l.pt')
#Model yolov5xu
#or model YOLOv8l
model.export(format='onnx')


100%|██████████| 83.7M/83.7M [00:00<00:00, 93.5MB/s]


Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8l summary (fused): 268 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs

PyTorch: starting from 'yolov8l.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (83.7 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 196.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.7s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.2 opset 17...
ONNX: export success ✅ 19.5s, saved as 'yolov8l.onnx' (166.8 MB)

Export complete (31.3s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8l.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8l.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8l.onnx'